In [1]:
import numpy as np
import matplotlib.pyplot as plt
import mxnet as mx
from mxnet import nd,gluon,autograd
from mxnet.gluon import nn

In [3]:
td = mx.gluon.data.vision.MNIST(train=True)
vd = mx.gluon.data.vision.MNIST(train=False)

In [4]:
type(td)

mxnet.gluon.data.vision.datasets.MNIST

In [5]:
def f_transform(data):
    return nd.moveaxis(data,2,0).astype('float32')/255

In [6]:
train_data = td.transform_first(f_transform)
val_data = vd.transform_first(f_transform)

In [7]:
batch_size = 50
tL = mx.gluon.data.DataLoader(train_data,shuffle=True,batch_size=batch_size)

In [8]:
vL = mx.gluon.data.DataLoader(val_data,shuffle=True,batch_size=batch_size)

In [9]:
model = nn.Sequential()
with model.name_scope():
    model.add(
        nn.Flatten(),
        nn.Dense(128,activation='relu'),
        nn.Dense(64,activation='relu'),
        nn.Dense(32,activation='relu'),
        nn.Dense(10,activation=None)
    )

In [10]:
ctx = mx.cpu(0) # mx.gpu(0)

In [12]:
model.initialize(mx.init.Xavier(),ctx)

In [13]:
trainer = gluon.Trainer(
    params = model.collect_params(),
    optimizer = 'sgd',
    optimizer_params={'learning_rate':0.01},
)

In [14]:
metric = mx.metric.Accuracy()
loss_function = gluon.loss.SoftmaxCrossEntropyLoss()

In [15]:
num_epochs = 5
for epoch in range(num_epochs):
    for X,y in tL:
        X = X.as_in_context(ctx)
        y = y.as_in_context(ctx)
        with autograd.record():
            y_hat = model(X)
            loss = loss_function(y_hat,y)
        loss.backward()
        metric.update(y,y_hat)
        trainer.step(batch_size=X.shape[0])
    name,acc = metric.get()
    print(acc)
    metric.reset()

0.7311666666666666
0.9058
0.9250666666666667
0.9361166666666667
0.9431833333333334


In [16]:
metric = mx.metric.Accuracy()
for Xv,yv in vL:
    Xv = Xv.as_in_context(ctx)
    yv = yv.as_in_context(ctx)
    metric.update(yv,model(Xv))
name,acc = metric.get()
print(acc)

0.948
